In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import nx_arangodb as nxadb
import os

from dotenv import load_dotenv
from arango import ArangoClient
from arango_datasets import Datasets



In [29]:
# Load environment variables from .env file
load_dotenv()

# Initialize the ArangoDB client.
host = os.getenv("HOST")
username = os.getenv("USER")
password = os.getenv("PASSWORD")


In [ ]:
# Connect to database
db = ArangoClient(hosts=host).db(username= username, password=password, verify=True)

# Connect to datasets
datasets = Datasets(db)

print(datasets.dataset_info("OPEN_INTELLIGENCE"))
dataset_name = "OPEN_INTELLIGENCE"

existing_collections = [col["name"] for col in db.collections() if not col["system"]]

if any(dataset_name in col for col in existing_collections):
    print(f"Dataset '{dataset_name}' already exists. Skipping reload.")
else:
    gdelt = datasets.load(dataset_name, preserve_existing=True)
    print("Dataset loaded successfully!")


In [ ]:
# Extract edge collection from the loaded dataset
edge_collection = "inRegion"  # Change this based on which edge collection you want

# Fetch edges from ArangoDB
edges = db.collection(edge_collection).all()

# Convert edges to a Pandas DataFrame
df = pd.DataFrame(edges)

# Ensure correct column names for NetworkX
df.rename(columns={"_from": "start_node", "_to": "end_node"}, inplace=True)

# Create Graph
G = nx.from_pandas_edgelist(df, "start_node", "end_node")

# Plot Graph
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
pos = nx.spring_layout(G, iterations=15, seed=1721)
fig, ax = plt.subplots(figsize=(15, 9))
nx.draw_networkx(G, pos=pos, ax=ax, **plot_options)

plt.show()

KeyboardInterrupt: 